## Setup parameters and libraries

In [1]:
import torch
import torch.nn as nn
import numpy as np
import albumentations as A
from albumentations.pytorch import ToTensorV2
import os
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
# from google.colab import drive

# drive.mount("/content/drive")

# TRAIN_IMG_PATH = "/content/drive/MyDrive/AI/Cityscape/images_prepped_train-20230811T065243Z-001/images_prepped_train"
# TRAIN_MASK_PATH = "/content/drive/MyDrive/AI/Cityscape/annotations_prepped_train-20230811T065240Z-001/annotations_prepped_train"
# VAL_IMG_PATH = "/content/drive/MyDrive/AI/Cityscape/images_prepped_test-20230811T065241Z-001"
# VAL_MASK_PATH = "/content/drive/MyDrive/AI/Cityscape/annotations_prepped_test-20230811T065240Z-001"

# EDA_DS_PATH = "/content/drive/MyDrive/AI/Advance CV - Project 3/Dataset"
# EDA_TRAIN_LABELS = os.path.join(EDA_DS_PATH, "train_labels.csv")
# EDA_TRAIN_FILES = os.path.join(EDA_DS_PATH, "train_files.csv")
# EDA_VAL_FILES = os.path.join(EDA_DS_PATH, "val_files.csv")

# MODEL_PATH = "/content/drive/MyDrive/AI/Advance CV - Project 3/models/FCN8s"

In [3]:
TRAIN_IMG_PATH = "./dataset/bootcamp/train/images"
TRAIN_MASK_PATH = "./dataset/bootcamp/train/annotations"
VAL_IMG_PATH = "./dataset/bootcamp/test/images"
VAL_MASK_PATH = "./dataset/bootcamp/test/annotations"

EDA_DS_PATH = "./dataset/bootcamp/train"
EDA_TRAIN_LABELS = os.path.join(EDA_DS_PATH, "train_labels.csv")
EDA_TRAIN_FILES = os.path.join(EDA_DS_PATH, "train_files.csv")
EDA_VAL_FILES = os.path.join(EDA_DS_PATH, "val_files.csv")

MODEL_PATH = "./models/FCN8s"

METRIC_PATH = "./metrics"

os.makedirs(MODEL_PATH, exist_ok=True)  # Ensure directory exist

In [4]:
files_df = pd.read_csv(EDA_TRAIN_FILES)
labels_df = pd.read_csv(EDA_TRAIN_LABELS)
val_files_df = pd.read_csv(EDA_VAL_FILES)

In [5]:
print(files_df.head())
print()
print(labels_df.head())
print()
train_files = files_df['filename'].to_numpy()
print(train_files[:5])
print()
val_files = val_files_df['filename'].to_numpy()
print(val_files[:5])

            filename
0  0001TP_006690.png
1  0001TP_006720.png
2  0001TP_006750.png
3  0001TP_006780.png
4  0001TP_006810.png

   class     label
0      0       sky
1      1  building
2      2      pole
3      3      road
4      4  sidewalk

['0001TP_006690.png' '0001TP_006720.png' '0001TP_006750.png'
 '0001TP_006780.png' '0001TP_006810.png']

['0016E5_07959.png' '0016E5_07961.png' '0016E5_07963.png'
 '0016E5_07965.png' '0016E5_07967.png']


## Dataset Loader

In [6]:
# refer to dataset.py
from dataset import CityScapes

In [7]:
train_datasets = CityScapes(TRAIN_IMG_PATH, TRAIN_MASK_PATH, train_files)
image, mask = train_datasets.__getitem__(0)
print(image)
print()
print(mask)
print()
print(image.shape)
print()
print(mask.shape)

val_datasets = CityScapes(VAL_IMG_PATH, VAL_MASK_PATH, val_files)

tensor([[[-1.8439, -1.9124, -1.8782,  ..., -1.8268, -1.8610, -1.8268],
         [-1.8782, -1.8782, -1.8953,  ..., -1.8268, -1.8610, -1.8268],
         [-1.8953, -1.8610, -1.8610,  ..., -1.9124, -1.8782, -1.8439],
         ...,
         [-1.8610, -1.8610, -1.8439,  ..., -1.7240, -1.6727, -1.6727],
         [-1.8439, -1.8439, -1.8439,  ..., -1.6898, -1.6213, -1.7240],
         [-1.8439, -1.8610, -1.8439,  ..., -1.7240, -1.7240, -1.7412]],

        [[-1.7556, -1.8256, -1.7906,  ..., -1.6681, -1.7031, -1.7206],
         [-1.7906, -1.7906, -1.8081,  ..., -1.6681, -1.7031, -1.7206],
         [-1.8081, -1.7731, -1.7731,  ..., -1.7556, -1.7206, -1.7381],
         ...,
         [-1.7556, -1.7556, -1.7381,  ..., -1.4580, -1.4055, -1.3880],
         [-1.7381, -1.7381, -1.7381,  ..., -1.4580, -1.3704, -1.4405],
         [-1.7556, -1.7556, -1.7381,  ..., -1.4580, -1.4405, -1.4580]],

        [[-1.5256, -1.5953, -1.5604,  ..., -1.4210, -1.4559, -1.4384],
         [-1.5604, -1.5604, -1.5779,  ..., -1

In [8]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_datasets, batch_size=32, shuffle=True, num_workers=0)
val_dataloader = DataLoader(val_datasets, batch_size=32, shuffle=True, num_workers=0)

## Model Architecture

In [9]:
from architecture import FCN8s

## Engine

In [10]:
from engine import DiceLoss
from engine import DiceCrossEntropyLoss
from engine import train

Z:\BEKA\2025\AI\Bootcamp\Source\Advanced CV\Project 3\engine.py:138: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  def train_engine(dataloader, model, loss_fn, optim, scaler=torch.cuda.amp.GradScaler()):


## Train FCN8s with DiceLoss

In [11]:
model = FCN8s(in_channels=3, out_channels=len(labels_df))
model.cuda()
loss_fn = DiceLoss()
optim = torch.optim.Adam(model.parameters(), lr = 1e-3)
print(model.out_channels)

12


In [12]:
model, logs = train(train_dataloader, val_dataloader, model, loss_fn, optim, 100, log_freq=1, save_best_model=True, 
              best_model_name='best_model_dice.pth', last_model_name='last_model_dice.pth', save_path=MODEL_PATH)
print(f"CUDA GPU: {torch.cuda.get_device_name(0)}")
print(f"Architecture: FCN8s DiceLoss")

100%|████████████████████████████████████████████████████████████████████████| 12/12 [00:13<00:00,  1.11s/it, loss=0.8]
Z:\BEKA\2025\AI\Bootcamp\Source\Advanced CV\Project 3\engine.py:184: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
100%|████████████████████████████████████████████████████████████████████████| 4/4 [00:01<00:00,  3.29it/s, loss=0.913]


Epoch 0/99
----------
Train Loss: 0.8637, Val Loss: 0.9121
✅ Best model saved! (Val Loss: 0.9121091067790985)



100%|████████████████████████████████████████████████████████████████████████| 4/4 [00:01<00:00,  3.41it/s, loss=0.913]


Epoch 1/99
----------
Train Loss: 0.7508, Val Loss: 0.9113
✅ Best model saved! (Val Loss: 0.911324605345726)



100%|█████████████████████████████████████████████████████████████████████████| 4/4 [00:01<00:00,  3.38it/s, loss=0.82]


...

Epoch 97/99
----------
Train Loss: 0.5417, Val Loss: 0.7092



100%|████████████████████████████████████████████████████████████████████████| 4/4 [00:01<00:00,  3.33it/s, loss=0.718]


Epoch 98/99
----------
Train Loss: 0.5404, Val Loss: 0.7159



100%|████████████████████████████████████████████████████████████████████████| 4/4 [00:01<00:00,  3.39it/s, loss=0.725]

Epoch 99/99
----------
Train Loss: 0.5393, Val Loss: 0.7180

Training complete in 17m 34s
Best test (smallest validation lost): 0.707434
CUDA GPU: NVIDIA GeForce RTX 4060 Laptop GPU
Architecture: FCN8s DiceLoss


In [13]:
logs_df = pd.DataFrame(logs)
logs_df.to_csv(os.path.join(METRIC_PATH, 'train_fcn8s_dice.csv'), index=False)

In [14]:
import torch.nn as nn

model = FCN8s(in_channels=3, out_channels=len(labels_df))
model.cuda()
loss_fn = nn.CrossEntropyLoss()
optim = torch.optim.Adam(model.parameters(), lr = 1e-3)
print(model.out_channels)

12


In [15]:
model, logs = train(train_dataloader, val_dataloader, model, loss_fn, optim, 100, log_freq=1, save_best_model=True, 
              best_model_name='best_model_cross.pth', last_model_name='last_model_cross.pth', save_path=MODEL_PATH)
print(f"CUDA GPU: {torch.cuda.get_device_name(0)}")
print(f"Architecture: FCN8s CrossEntropyLoss")

100%|█████████████████████████████████████████████████████████████████████████| 4/4 [00:01<00:00,  3.36it/s, loss=2.31]


Epoch 0/99
----------
Train Loss: 1.5918, Val Loss: 2.2300
✅ Best model saved! (Val Loss: 2.2300477623939514)



100%|█████████████████████████████████████████████████████████████████████████| 4/4 [00:01<00:00,  3.50it/s, loss=2.74]


Epoch 1/99
----------
Train Loss: 0.9785, Val Loss: 2.6054



100%|█████████████████████████████████████████████████████████████████████████| 4/4 [00:01<00:00,  3.45it/s, loss=0.99]


...

Epoch 97/99
----------
Train Loss: 0.1006, Val Loss: 0.4146



100%|████████████████████████████████████████████████████████████████████████| 4/4 [00:01<00:00,  3.55it/s, loss=0.446]


Epoch 98/99
----------
Train Loss: 0.0979, Val Loss: 0.4017



100%|████████████████████████████████████████████████████████████████████████| 4/4 [00:01<00:00,  3.42it/s, loss=0.362]

Epoch 99/99
----------
Train Loss: 0.0966, Val Loss: 0.3999

Training complete in 16m 32s
Best test (smallest validation lost): 0.323392
CUDA GPU: NVIDIA GeForce RTX 4060 Laptop GPU
Architecture: FCN8s CrossEntropyLoss


In [16]:
logs_df = pd.DataFrame(logs)
logs_df.to_csv(os.path.join(METRIC_PATH, 'train_fcn8s_cross.csv'), index=False)

In [17]:
import torch.nn as nn

model = FCN8s(in_channels=3, out_channels=len(labels_df))
model.cuda()
loss_fn = DiceCrossEntropyLoss()
optim = torch.optim.Adam(model.parameters(), lr = 1e-3)
print(model.out_channels)

12


In [18]:
model, logs = train(train_dataloader, val_dataloader, model, loss_fn, optim, 100, log_freq=1, save_best_model=True, 
              best_model_name='best_model_combined.pth', last_model_name='last_model_combined.pth', save_path=MODEL_PATH)
print(f"CUDA GPU: {torch.cuda.get_device_name(0)}")
print(f"Architecture: FCN8s DiceLoss+CrossEntropyLoss")

100%|█████████████████████████████████████████████████████████████████████████| 4/4 [00:01<00:00,  3.42it/s, loss=4.55]


Epoch 0/99
----------
Train Loss: 1.1933, Val Loss: 4.5443
✅ Best model saved! (Val Loss: 4.544333100318909)



100%|█████████████████████████████████████████████████████████████████████████| 4/4 [00:01<00:00,  3.33it/s, loss=2.42]


Epoch 1/99
----------
Train Loss: 0.8515, Val Loss: 2.4569
✅ Best model saved! (Val Loss: 2.456887185573578)



100%|████████████████████████████████████████████████████████████████████████| 4/4 [00:01<00:00,  3.30it/s, loss=0.881]


...

Epoch 97/99
----------
Train Loss: 0.1580, Val Loss: 0.4318



100%|████████████████████████████████████████████████████████████████████████| 4/4 [00:01<00:00,  3.44it/s, loss=0.431]


Epoch 98/99
----------
Train Loss: 0.1566, Val Loss: 0.4233



100%|████████████████████████████████████████████████████████████████████████| 4/4 [00:01<00:00,  3.48it/s, loss=0.476]

Epoch 99/99
----------
Train Loss: 0.1562, Val Loss: 0.4393

Training complete in 16m 53s
Best test (smallest validation lost): 0.356853
CUDA GPU: NVIDIA GeForce RTX 4060 Laptop GPU
Architecture: FCN8s DiceLoss+CrossEntropyLoss


In [19]:
logs_df = pd.DataFrame(logs)
logs_df.to_csv(os.path.join(METRIC_PATH, 'train_fcn8s_combined.csv'), index=False)